In [1]:
# class JSONTable:
#     # creates a class with variables dynamically defined by the json input
#     def __init__(self, schema):
#         self.schema = schema
#         self.key_name = schema.keys[0]
#         # while next level is a list
#         # generate new multi levelcolumn name
#         # else append to last column
#         self.colnames = ""


# schema = {"company": [{"kpi_name": ["period_name", "value"]}]}

# data_class = JSONTable(schema)

# data = {"tgt": [{"net_revenue": ["Q1_2020", "4500"]}]}

# data_class.add_record(data)
# data_class.add_records([data, data])

# data_class.return_panel_data()  # this returns a dataframe for saving in sql / csv
# data_class.json_dump(
#     show_schema=True
# )  # this dumps the json show schema has schema as first record


In [107]:
import pandas as pd

# d = {'key': ['foo', 'foo', 'foo', 'foo', 'bar', 'bar', 'bar', 'bar', 'crow', 'crow', 'crow', 'crow'], 
#      'date': ['2021-01-01', '2021-01-01', '2021-01-02', '2021-01-02', '2021-01-01', '2021-01-01','2021-01-02', '2021-01-02', '2021-01-01', '2021-01-01', '2021-01-02', '2021-01-02'], 
#      'class': [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2],
#      'count': [12, 3, 5, 5, 3, 1, 4, 1, 7, 3, 8, 2],
#      'percent': [.8, .2, .5, .5, .75, .25, .8, .2, .7, .3, .8, .2]
# }
# df = pd.DataFrame(data=d)
# df.to_csv("data.csv",index = False)
df = pd.read_csv('data.csv')
# d   = {'count': ('count', 'sum'), 'predictions': ('percent', list)}
# g   = df.groupby(['key', 'date']).agg(**d).groupby(level=0).agg(list)
# dct = [{'key': k, **v} for k, v in g.to_dict(orient='index').items()]

In [108]:
df

,ticker,sector,kpi_name,kpi_meta_mapping
0,TGT,consumer,net_revenue,revenue_dashboard
1,TGT,consumer,internet_sales,internet_dashboard
2,TGT,consumer,DSO,dso_dashboard
3,WMT,consumer,net_revenue,revenue_dashboard
4,WMT,consumer,DSO,dso_dashboard
5,AMZN,consumer,net_revenue,revenue_dashboard
6,AMZN,consumer,DSO,dso_dashboard
7,AMZN,consumer,internet_sales,internet_dashboard
8,AMZN,Internet_retail,net_revenue,revenue_dashboard
9,AMZN,Internet_retail,DSO,dso_dashboard


In [10]:
dctt6

[{'key': 'bar', 'count': [4, 5], 'predictions': [[0.75, 0.25], [0.8, 0.2]]},
 {'key': 'crow', 'count': [10, 10], 'predictions': [[0.7, 0.3], [0.8, 0.2]]},
 {'key': 'foo', 'count': [15, 10], 'predictions': [[0.8, 0.2], [0.5, 0.5]]}]

In [98]:
d = {'ticker': ['tgt', 'tgt', 'wmt', 'wmt', 'amzn', 'amzn'], 
     'sector': ['consumer', 'consumer', 'consumer', 'consumer', 'consumer', 'custom_3'], 
     'kpi': ['net_revenue', 'other', 'net_revenue', 'other', 'net_revenue', 'other'],
     'kpi_dashboard': ['revenue_dashboard', 'custom_dashboard', 'revenue_dashboard', 'other', 'revenue_dashboard', 'other']
}
df = pd.DataFrame(data=d)
df.set_index(['ticker','sector'],inplace=True)

In [99]:
df

kpi      kpi_dashboard
ticker sector                                  
tgt    consumer  net_revenue  revenue_dashboard
       consumer        other   custom_dashboard
wmt    consumer  net_revenue  revenue_dashboard
       consumer        other              other
amzn   consumer  net_revenue  revenue_dashboard
       custom_3        other              other

In [104]:
data = []
for ticker, df1 in df.groupby(level=0):
    data.append({'ticker': ticker, 'sector': []})
    for sector, df2 in df1.groupby(level=1):
        data[-1]['sector'].append({'sector': sector, 'kpis': df2.to_dict('records')})
import json
print(json.dumps(data, indent=2))

[
  {
    "ticker": "amzn",
    "sector": [
      {
        "sector": "consumer",
        "kpis": [
          {
            "kpi": "net_revenue",
            "kpi_dashboard": "revenue_dashboard"
          }
        ]
      },
      {
        "sector": "custom_3",
        "kpis": [
          {
            "kpi": "other",
            "kpi_dashboard": "other"
          }
        ]
      }
    ]
  },
  {
    "ticker": "tgt",
    "sector": [
      {
        "sector": "consumer",
        "kpis": [
          {
            "kpi": "net_revenue",
            "kpi_dashboard": "revenue_dashboard"
          },
          {
            "kpi": "other",
            "kpi_dashboard": "custom_dashboard"
          }
        ]
      }
    ]
  },
  {
    "ticker": "wmt",
    "sector": [
      {
        "sector": "consumer",
        "kpis": [
          {
            "kpi": "net_revenue",
            "kpi_dashboard": "revenue_dashboard"
          },
          {
            "kpi": "other",
            "kpi_dash

In [69]:
import pandas as pd
import json

df = pd.DataFrame(
    data=[
        ("EURO","UK",100),
        ("EURO","Ireland",80),
        ("AMR","Mexico",20),
        ("AMR","California",110),
        ("APAC","Japan",90)
    ], 
    columns=["region","country","total_count"]
)

#First get the regions into a list of dictionary objects
regions = df[["region", "total_count"]].groupby("region").sum()
regions["geo"] = regions.index.values
regions = regions.to_dict(orient="records") 

#now add the countries to each region dictionary
for region in regions:
    countries = df[df["region"] == region["geo"]].drop("region", axis=1).groupby("country").sum()
    countries["geo"] = countries.index.values
    region["drill_through"] = countries.to_dict(orient="records")
    
#Serialize the list of regions as JSON
json_str = json.dumps(regions)

print(json_str)

[{"total_count": 130, "geo": "AMR", "drill_through": [{"total_count": 110, "geo": "California"}, {"total_count": 20, "geo": "Mexico"}]}, {"total_count": 90, "geo": "APAC", "drill_through": [{"total_count": 90, "geo": "Japan"}]}, {"total_count": 180, "geo": "EURO", "drill_through": [{"total_count": 80, "geo": "Ireland"}, {"total_count": 100, "geo": "UK"}]}]


In [73]:
df.groupby('region').apply(lambda x: x.to_json(orient='records'))

region
AMR     [{"region":"AMR","country":"Mexico","total_cou...
APAC    [{"region":"APAC","country":"Japan","total_cou...
EURO    [{"region":"EURO","country":"UK","total_count"...
dtype: object

In [38]:
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

In [80]:

RenderJSON(str)